<a href="https://colab.research.google.com/github/metamorphic-rock/Time_series_Forecasting_UsingFacebookProphet/blob/main/Manila_WithFeatureSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model With Feature Selection**

## Importing Dependencies

In [ ]:
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


creating a dataframe from the swg csv file

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SWG data/Manila Data.csv')
df = df.set_index('Year')
df.index = pd.to_datetime(df.index)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.columns

# Correlation between SWG and Socioeconomic Factors

In [ ]:
#get correlations of each features in dataset
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
corrmat


In [ ]:
corrmat.to_csv('corellation.csv', index=True)
from google.colab import files

# Download the file
files.download('corellation.csv')



# Feature Selection


In [ ]:
df =df[['Year index', 'SWG ', 'Population', 'Death rate', 'Sex ratio','Average price of Gasoline', 'Average price of Diesel', 'Number of Marriages','Labor Force Participation','Average Age','Average Relative Humidity','Number of tropical cyclones ']]

In [ ]:
df.columns

In [ ]:
df = df.rename(columns={'Year index': 'ds', 'SWG ': 'y'})

In [ ]:
df['ds']=pd.to_datetime(df['ds']) #converting the 'ds' column into datetime format


In [ ]:
df.plot(x='ds',y='y',figsize=(18,6))
plt.show()

## Splitting the data

In [ ]:
train = df.loc[df.index < '2020'] #All Data before 2020 will be used as the train dataset
test = df.loc[df.index >= '2020']


In [ ]:
train

In [ ]:
test

## Creating and training the model

In [ ]:

ProphetModel=Prophet()
def AddFeatures(df):
  for i in df.columns:
    ProphetModel.add_regressor(i)
  return ProphetModel

In [ ]:
ProphetModel.fit(train)

## Forecasting MSWG

In [ ]:
future=ProphetModel.make_future_dataframe(periods=7, freq = 'Y') #Forecasting 7 years of MSWG from 2019

In [ ]:

future.head(5)

In [ ]:
forecast=ProphetModel.predict(future)

In [ ]:
forecast.tail()

In [ ]:
forecast[['ds','yhat','yhat_lower','yhat_upper']]

In [ ]:
ProphetModel.plot(forecast)

In [ ]:
plot_plotly(ProphetModel,forecast)

In [ ]:
plot_components_plotly(ProphetModel, forecast)

## Evaluating the model

In [ ]:
Predictions=forecast.iloc[-2:]['yhat']

In [ ]:
from statsmodels.tools.eval_measures import rmse,rmspe,meanabs
print("Mean Absolute Error between actual and  predicted values: ",meanabs(Predictions,test['y']))
print("Root Mean Squared Error between actual and  predicted values: ",rmse(Predictions,test['y']))
print("Root Mean Squared Percentage Error between actual and  predicted values: ",rmspe(Predictions,test['y']))
print("Mean Value of Test Dataset:", test['y'].mean())

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
print("Mean Absolute Percentage Error between actual and  predicted values: ",mean_absolute_percentage_error(test['y'],Predictions))

More Plots

In [ ]:

forecast_data_orig = forecast # make sure we save the original forecast data
forecast_data_orig['yhat'] = np.exp(forecast_data_orig['yhat'])
forecast_data_orig['yhat_lower'] = np.exp(forecast_data_orig['yhat_lower'])
forecast_data_orig['yhat_upper'] = np.exp(forecast_data_orig['yhat_upper'])

ProphetModel.plot(forecast_data_orig)